# Scraping

In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
import time

In [2]:
# Define path
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}

### NASA Mars News

In [ ]:
# Instantiate a splinter browser
browser = Browser('chrome', **executable_path, headless=False)

# Define Url for the browser to launch
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(nasa_url)
time.sleep(2)

# HTML object
nasa_html = browser.html
# Parse HTML with BS
nasa_soup = bs(nasa_html, 'html.parser')
# Retrieve all elements that contain quotes information
content_title = nasa_soup.find_all('div', class_='content_title')
article_teaser = nasa_soup.find_all('div', class_='article_teaser_body')

# Shut the browser down when done!
browser.quit()

In [ ]:
# Manually grabbing the text...(?)
news_title = content_title[1].text
news_p = article_teaser[0].text

### JPL Mars Space Images - Featured Image

In [ ]:
# Instantiate a splinter browser
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
time.sleep(2)

# HTML object
jpl_html = browser.html
# Parse HTML with BS
jpl_soup = bs(jpl_html, 'html.parser')
# Retrieve img path
img_path = jpl_soup.find_all('img')[3]['src']



### I CAN'T FIGURE OUT ### NOTHING WORKS ### 😭😭😭😭😭😭😭😭😭😭
# Click the imgae to go to img page
# browser.click_link_by_partial_text('thumb')

# browser.click_link_by_href(img_path)
# browser.attach_file('img', img_path)
# stackoverflow = browser.find_element_by_xpath(img_path).click()

# Can I just skip the first url



# Shut the browser down when done!
browser.quit()

In [ ]:
img_url = img_path
print(img_url)

In [ ]:
# Grabbed the URL
featured_image_url =

### Mars Facts

In [ ]:
# Automatically scrape any tabular data from a page w pandas
mars_url = 'https://space-facts.com/mars/'
tables = pd.read_html(mars_url)

In [ ]:
# Fishing out wanted table, and transpose it
# df = tables[0].T
# df.head()

In [ ]:
# # Raneme the column
# df = df.rename(columns={0: 'Equatorial Diamete',
#                        1: 'Polar Diameter',
#                        2: 'Mass',
#                        3: 'Moons',
#                        4: 'Orbit Distance',
#                        5: 'Orbit Period',
#                        6: 'Surface Temperature',
#                        7: 'First Record',
#                        8: 'Recorded By'})

In [ ]:
# # Removed the first row
# df = df.iloc[1:] 
# df

In [ ]:
# Fishing out wanted table, and transpose it
df = tables[0]

In [ ]:
# Raneme the column
df = df.rename(columns={0: 'Description',
                       1: 'Mars'})

In [ ]:
# Set the index to the `abb` column
df.set_index('Description', inplace=True)
df

In [ ]:
# Generate HTML tables from DataFrames
html_table = df.to_html()

In [ ]:
# Convert df to html
df.to_html('mars_table.html')

# Mars Hemispheres

In [7]:
# Instantiate a splinter browser
browser = Browser('chrome', **executable_path, headless=False)


# Define URL to scrape title & url
marshem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(marshem_url)
time.sleep(2)

# HTML object
marshem_html = browser.html
# Parse HTML with BS
marshem_soup = bs(marshem_html, 'html.parser')
# Retrieve img path
description_class = marshem_soup.find_all('div', class_='description')

In [ ]:
# Create an empty list to store dictionaries of imag_url & title
hemisphere_image_urls = []

# Run loop to get the info
for dsc in description_class:
    # Create an empty dictionary to store img_url & title
    hemisphere_dict = {}
    # GET the title into dictionary
    hemisphere_dict['title'] = dsc.a.h3.text
    # Get image URL
    hemisphere_dict['img_url'] = 😭😭😭😭😭😭😭😭😭😭
    # Add the dictionary to the list 
    hemisphere_image_urls.append(hemisphere_dict)

# Shut the browser down when done!
browser.quit()

In [ ]:
hemisphere_image_urls

In [8]:
##### DRAFT HERE ######## Getting IMAGE_URL
# URL of page to be scraped ----- This is too manaual, it ain't right...
## I can't figure out how to use splinter for it to click on the image T^T ##


for dsc in description_class:
    #PART 1
    # Retrieve img path
    img_dwnload = dsc.find('a')['href']
    
        # Click the image......
        #some code here to make chromebrowser click the image
        
        
        # Once image is clicked, and browser goes to another page:
        ## SEE BELOW PART 2
        
        
        
        
        
        
    
    
        # CODE FROM CLASS, THERE'S NO 'NEXT'..... 
        #Click the 'Next' button on each page
        try:
        browser.click_link_by_partial_text('next')
 
        except:
        print("Scraping Complete")
    
#     print(img_dwnload)
        

# Shut the browser down when done!
browser.quit()

/search/map/Mars/Viking/cerberus_enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
/search/map/Mars/Viking/valles_marineris_enhanced


In [9]:
# Instantiate a splinter browser
browser = Browser('chrome', **executable_path, headless=False)


# Define URL to scrape title & url
testurl = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
browser.visit(testurl)
time.sleep(2)

# HTML object
test_html = browser.html
# Parse HTML with BS
testsoup = bs(test_html, 'html.parser')

## PART 2

# Retrieve img path
test_class = testsoup.find_all('div', class_='downloads')

for test in test_class:
    # Retrieve img path
    test_li = test.find('li')
    a = test_li.find('a')['href']
    print(a)

# Shut the browser down when done!
browser.quit()

https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg


### Jupyter Notebook Ended.

In [ ]:
# Converting your Jupyter notebook into a Python script called scrape_mars.py with a function called scrape
# ................... HOW?????